In [1]:
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

## Указать все пути
MODEL_NAME - путь к предобученной модели (уже указан)  
TEXT_FILES_DIR - путь к директории с текстами  
FAISS_INDEX_PATH - путь для сохранения/загрузки FAISS-индекса  
FILENAMES_PATH - путь для сохранения/загрузки списка имен файлов

In [2]:
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
TEXT_FILES_DIR = "garant"
FAISS_INDEX_PATH = "results_BERT_Default/text_index.faiss"
FILENAMES_PATH = "results_BERT_Default/filenames.npy"

In [3]:
model = SentenceTransformer(MODEL_NAME)
index = faiss.read_index(FAISS_INDEX_PATH)
filenames = np.load(FILENAMES_PATH, allow_pickle=True)

## Функция разбиения длинных текстов на чанки

In [4]:
def split_text_by_length(text, max_length=1000):
    words = text.split()
    chunks = []
    current_chunk = ""
    for word in words:
        if len(current_chunk) + len(word) + 1 <= max_length:
            current_chunk = current_chunk + " " + word if current_chunk else word
        else:
            chunks.append(current_chunk)
            current_chunk = word
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

## Функция для вычисления эмбеддинга с учётом разбиения длинного текста

In [5]:
def get_document_embedding(text, model, chunk_size=1000):
    if len(text) <= chunk_size:
        return model.encode(text)
    else:
        chunks = split_text_by_length(text, max_length=chunk_size)
        chunk_embeddings = model.encode(chunks, convert_to_numpy=True)
        return np.mean(chunk_embeddings, axis=0)

## Функция поиска похожих текстов

In [6]:
def find_similar_texts_from_file(file_path, top_k=5):
    with open(file_path, "r", encoding="utf-8") as f:
        query_text = f.read().strip()

    query_embedding = model.encode([query_text], convert_to_numpy=True)
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)

    distances, indices = index.search(query_embedding.astype('float32'), top_k) # поиск ближайших соседей в нормализованных эмбеддингах

    print("Наиболее похожие тексты:")
    for i, idx in enumerate(indices[0]):
        text_filename = filenames[idx]
        similarity = max(0, (1 - distances[0][i] / 2)) * 100  
        print(f"{i+1}. {text_filename} (схожесть: {similarity:.2f}%)")

        found_file_path = os.path.join(TEXT_FILES_DIR, text_filename)
        if os.path.exists(found_file_path):
            with open(found_file_path, "r", encoding="utf-8") as f:
                preview_text = f.read(300)
            print(preview_text + "...")
        else:
            print("[Файл не найден в папке]")
        print("-" * 80)

## Тестирование
file_path - путь к текстовому файлу, который будет поступать на вход

In [7]:
file_path = "garant.txt"

In [8]:
find_similar_texts_from_file(file_path, top_k=20)

Наиболее похожие тексты:
1. garant_1261.txt (схожесть: 100.00%)
﻿
Постановление Мэра г. Хабаровска
от 11 ноября 2004 г. N 1650
"О проведении Международного дня инвалидов в городе Хабаровске в 2004 году"

В связи с проведением 3 декабря 2004 года Международного дня инвалидов и в целях привлечения внимания широких слоев общественности города, предприятий, учрежден...
--------------------------------------------------------------------------------
2. garant_1425.txt (схожесть: 71.65%)
﻿
Постановление Главы города Челябинска 
от 8 июня 2004 г. N 991-п 
"Об утверждении "Программы мероприятий, обеспечивающих доступ инвалидов и
маломобильных групп населения в существующие здания общественного 
назначения" 

В целях реализации постановления Главы города от 18.02.2003г. N 240-п "О бесп...
--------------------------------------------------------------------------------
3. garant_0078.txt (схожесть: 70.16%)
﻿
Постановление Мэра г. Хабаровска
от 12 ноября 2004 г. N 1662
"О городском смотре-конкурс